In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv
/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv
/kaggle/input/sentiment-analysis-dataset/training.1600000.processed.noemoticon.csv
/kaggle/input/sentiment-analysis-dataset/train.csv
/kaggle/input/sentiment-analysis-dataset/testdata.manual.2009.06.14.csv
/kaggle/input/sentiment-analysis-dataset/test.csv
/kaggle/input/sentiment-analysis-for-financial-news/all-data.csv
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/Sentences_66Agree.txt
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/Sentences_AllAgree.txt
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/README.txt
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/License.txt
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/Sentences_75Agree.txt
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/Sentences_50Agree

BERT vs RoBERTa Sentiment Analysis: https://www.kaggle.com/code/ludovicocuoghi/twitter-sentiment-analysis-with-bert-vs-roberta#BERT-Sentiment-Analysis
(very similar to what we need to do, maybe we can just choose either one, BERT or RoBERTa)



In [2]:
train_data = pd.read_csv('/kaggle/input/sentiment-analysis-for-financial-news/all-data.csv',encoding='unicode_escape',names=['Sentiment', 'Text']);
test_data = pd.read_csv('/kaggle/input/sentiment-analysis-dataset/test.csv',encoding='latin1');

In [3]:
train_data.head()

,Sentiment,Text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [4]:
test_data.head()

,textID,text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,morning,0-20,Afghanistan,38928346.0,652860.0,60.0
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,noon,21-30,Albania,2877797.0,27400.0,105.0
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,night,31-45,Algeria,43851044.0,2381740.0,18.0
3,01082688c6,happy bday!,positive,morning,46-60,Andorra,77265.0,470.0,164.0
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,noon,60-70,Angola,32866272.0,1246700.0,26.0


In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4846 entries, 0 to 4845
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sentiment  4846 non-null   object
 1   Text       4846 non-null   object
dtypes: object(2)
memory usage: 75.8+ KB


In [6]:
# train_data.drop("textID", axis=1)

In [7]:
from transformers import BertTokenizerFast
from transformers import TFBertModel
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel

2024-03-28 18:17:40.832199: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-28 18:17:40.832322: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-28 18:17:40.965394: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
train_data = train_data[['Text', 'Sentiment']]

In [9]:
test_data = test_data[['text', 'sentiment']]

In [10]:
train_data.head()

,Text,Sentiment
0,"According to Gran , the company has no plans t...",neutral
1,Technopolis plans to develop in stages an area...,neutral
2,The international electronic industry company ...,negative
3,With the new production plant the company woul...,positive
4,According to the company 's updated strategy f...,positive


In [11]:
test_data.head()

,text,sentiment
0,Last session of the day http://twitpic.com/67ezh,neutral
1,Shanghai is also really exciting (precisely -...,positive
2,"Recession hit Veronique Branquinho, she has to...",negative
3,happy bday!,positive
4,http://twitpic.com/4w75p - I like it!!,positive


BERT

In [12]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast as BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
from keras.callbacks import EarlyStopping
from torch import nn

In [13]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Load the dataset
data = pd.read_csv('/kaggle/input/sentiment-analysis-for-financial-news/all-data.csv',encoding='unicode_escape',names=['Sentiment', 'Text'])

# Split the dataset into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(data['Text'], data['Sentiment'], test_size=0.2)

# Convert sentiment labels to numerical values
sentiment_mapping = {'positive': 1, 'negative': 0, 'neutral' : 0.5}  # adjust this to match your labels
train_labels = train_labels.map(sentiment_mapping).astype(int)
val_labels = val_labels.map(sentiment_mapping).astype(int)

# Ensure all text data are strings
train_texts = train_texts.astype(str)
val_texts = val_texts.astype(str)

# Remove any rows with 'None' values
train_texts = train_texts.dropna()
val_texts = val_texts.dropna()

criterion = nn.CrossEntropyLoss()


In [14]:
# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the datasets
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, return_tensors='pt')
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, return_tensors='pt')

# Create a PyTorch Dataset
class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Create instances of the Dataset for training and validation
train_dataset = SentimentDataset(train_encodings, train_labels.values)
val_dataset = SentimentDataset(val_encodings, val_labels.values)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [15]:
# Initialize the BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
# Move the model to the device
model = model.to(device)

# Initialize the optimizer
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)

# Create a DataLoader for the training set
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)
epochs = 3

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# Train the model
for epoch in range(epochs):
    model.train()  # Ensure the model is in training mode
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
    # Evaluation
    model.eval()  # Put the model in evaluation mode
    correct_predictions = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

    val_accuracy = correct_predictions.double() / len(val_dataset)
    print(f'Validation Accuracy: {val_accuracy:.4f}')


100%|██████████| 243/243 [01:00<00:00,  4.04it/s]


Validation Accuracy: 0.8742


100%|██████████| 243/243 [00:59<00:00,  4.10it/s]


Validation Accuracy: 0.8897


100%|██████████| 243/243 [00:59<00:00,  4.09it/s]


Validation Accuracy: 0.8825


In [17]:
print("debug")

debug


In [18]:
test_df = pd.read_csv('/kaggle/input/sentiment-analysis-for-financial-news/all-data.csv',encoding='latin1')
test_df.head()

,neutral,"According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing ."
0,neutral,Technopolis plans to develop in stages an area...
1,negative,The international electronic industry company ...
2,positive,With the new production plant the company woul...
3,positive,According to the company 's updated strategy f...
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...


In [19]:
#test_df = pd.read_csv('/kaggle/input/sentiment-analysis-dataset/test.csv',encoding='latin1')
test_df = pd.read_csv('/kaggle/input/sentiment-analysis-for-financial-news/all-data.csv',encoding='latin1', names=['sentiment', 'text'])
#test_df = pd.read_csv('/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv',encoding='latin1')
test_df = test_df[['text', 'sentiment']]
train_texts, val_texts, train_labels, val_labels = train_test_split(test_df['text'], test_df['sentiment'], test_size=0.99)
train_labels = train_labels.map(sentiment_mapping)
val_labels = val_labels.map(sentiment_mapping)
train_texts = train_texts.astype(str)
val_texts = val_texts.astype(str)

# Remove any rows with 'None' values
train_texts = train_texts.dropna()
val_texts = val_texts.dropna()

train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, return_tensors='pt')
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, return_tensors='pt')

train_dataset = SentimentDataset(train_encodings, train_labels.values)
val_dataset = SentimentDataset(val_encodings, val_labels.values)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)


def predict(model, dataloader):
    model.eval()  # Put the model in evaluation mode
    predictions = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            predictions.extend(preds.cpu().numpy())
    return predictions


In [20]:
import torch
from tqdm import tqdm

def evaluate(model, val_loader, val_dataset):
    correct_predictions = 0
    total_samples = len(val_dataset)
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs.logits, dim=1)
            correct_predictions += torch.sum(preds == labels).item()

    val_accuracy = correct_predictions / total_samples
    print(f'Validation Accuracy: {val_accuracy:.4f}')
    return val_accuracy

# Example usage
# Assuming model, val_loader, and val_dataset are defined

# Run prediction
predictions = predict(model, val_loader)

# Evaluate
accuracy = evaluate(model, val_loader, val_dataset)
print("Test Predictions:", predictions)
print("Test Accuracy:", accuracy)

Evaluating: 100%|██████████| 300/300 [00:22<00:00, 13.48it/s]

Validation Accuracy: 0.3872
Test Predictions: [1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,